In [4]:
import os
import scanpy as sc
import numpy as np
import pandas as pd
import torch
import yaml
from celldreamer.paths import ROOT

from celldreamer.estimator.celldreamer_estimator import CellDreamerEstimator
from celldreamer.paths import DATA_DIR
from celldreamer.data.utils import Args

from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.model_summary import ModelSummary

from scvi.distributions import NegativeBinomial
import torch.nn.functional as F

/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [2]:
def tensor2adata(X, annot=None):
    if annot is None:
        adata = sc.AnnData(X=X.numpy())
    else:
        adata = sc.AnnData(X=X.numpy(),
                       obs=annot)
    sc.tl.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    return adata

Read config

In [3]:
config = yaml.safe_load(open(ROOT / "configs/scrna_seq/config.yaml", 
                            "rb"))
args = Args(config["args"]) 

Initialize data

In [4]:
estimator = CellDreamerEstimator(args)

Create the training folders...
Initialize data module...


/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


KeyError: 'X_norm'

In [ ]:
# estimator.train()

In [ ]:
estimator.generative_model.load_state_dict(torch.load("/nfs/homedirs/pala/celldreamer/project_dir/try_experiment_eb/checkpoints/epoch_132.ckpt")["state_dict"])

Collect the batches 

In [ ]:
real_data = []
annot = []
 
for batch in estimator.datamodule.train_dataloader:
    real_data.append(batch["X"])
    annot.append(batch["y"]["y_experimental_time"])

real_data = torch.cat(real_data, dim=0) 
annot = pd.DataFrame(torch.cat(annot))
annot.columns = ["experimental_time"]

In [ ]:
adata_real = tensor2adata(real_data, annot)

In [ ]:
sc.pl.umap(adata_real, color="experimental_time")

Sampling

In [ ]:
fake_data = []
zs = []

for i in range(20):
    sample, z = estimator.generative_model.sample(20, 
                                       n_sample_steps=100, 
                                       clip_samples=False, library_size=1e3)
    fake_data.append(sample.to("cpu"))
    zs.append(z.to("cpu"))

In [ ]:
fake_data = torch.cat(fake_data, dim=0)

In [ ]:
fake_data.sum(1)

In [ ]:
adata_fake = tensor2adata(fake_data, annot=None)

In [ ]:
sc.pl.umap(adata_fake)

In [ ]:
z

Match matrices together

In [ ]:
real_and_fake_data = torch.cat([real_data, fake_data], dim=0)

In [ ]:
real_and_fake_data

In [ ]:
annot = ["true" for _ in range(len(real_data))] + ["fake" for _ in range(len(fake_data))]
annot = pd.DataFrame(annot)
annot.columns = ["Dataset"]
adata_real_and_fake = tensor2adata(real_and_fake_data, annot)

In [ ]:
sc.pl.umap(adata_real_and_fake, color="Dataset")

In [ ]:
fake_data.sum(1).max()

In [ ]:
for i in fake_data.sum(1):
    print(i)

Check encoder training

In [ ]:
fake_data_from_z0 = []
annot_from_z0 = []

for batch in estimator.datamodule.train_dataloader:
    library_size = batch["X"].sum(1)
    z0 = estimator.generative_model.z0_from_x(estimator.generative_model._scale_batch(batch["X"]))
    z0_softmax = F.softmax(z0, dim=1)
    distr = NegativeBinomial(mu=library_size.unsqueeze(1)*z0_softmax, 
                             theta=torch.exp(estimator.generative_model.theta))
    fake_data_from_z0.append(distr.sample())
    annot_from_z0.append(batch["y"]["y_experimental_time"])

fake_data_from_z0 = torch.cat(fake_data_from_z0, dim=0) 
annot_from_z0 = pd.DataFrame(torch.cat(annot_from_z0))
annot_from_z0.columns = ["experimental_time"]

In [ ]:
fake_data_from_z0

In [ ]:
estimator.generative_model.z0_from_x

In [ ]:
estimator.generative_model.z0_from_x(estimator.generative_model._scale_batch(batch["X"]))

In [ ]:
F.softmax(estimator.generative_model.z0_from_x(torch.log(1+batch["X"])), dim=1)

In [ ]:
real_and_fake_data = torch.cat([real_data, fake_data_from_z0], dim=0)

In [ ]:
annot = ["true" for _ in range(len(real_data))] + ["fake" for _ in range(len(fake_data_from_z0))]
annot = pd.DataFrame(annot)
annot.columns = ["Dataset"]
adata_real_and_fake = tensor2adata(real_and_fake_data, annot)

In [ ]:
sc.pl.umap(adata_real_and_fake, color="Dataset")

In [ ]:
adata_real_and_fake.X.sum(1).mean()

In [ ]:
adata_real.X.sum(1)

In [5]:
adata = sc.read_h5ad("/home/icb/alessandro.palma/environment/celldreamer/datasets/scrnaseq_dataset/eb_phate.h5ad")

/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
